# Airlines Delay

### Project developed by:
- Adam Nogueira (up202007519)
- Eduardo Silva (up202004999)
- João Félix (up202008867)

## Table of Contents



## Introduction
This project involves developing a data mining case study, which is described in a separate document provided on Moodle. The main focus of the project is a predictive data mining task, the details of which are outlined in the case study description.




### Bibliography
NumPy Developers, Numpy documentation, URL: https://numpy.org/doc/stable/user/index.html#user <br>
pandas development team, pandas documentation, URL: https://pandas.pydata.org/docs/user_guide/index.html#user-guide<br>
Matplotlib Development team, Matplotlib documentation, URL: https://matplotlib.org/stable/index.html <br>
scikit-learn developers, scikit-learn documentation, URL: https://scikit-learn.org/0.18/documentation.html<br>

### Approach

The approach to this project was done as follows:

1. **Data analysis**: First we analyzed the dataset to inspect for the need for data pre-processing: checked the corresponding histograms, class distribution, and the existence of missing or null values.
2. **Algorithm implementation**: Flowing that, we defined the training and test sets using train/test split, resampled the dataset, and applied the SciKit Learn's algorithms to obtain the first results.
3. **Evaluation and refinement**: After analyzing the first results, tunning of each algorithm was done utilizing the SciKit Learn GridSearchCV to find the parameters of each algorithm that yielded the best overall results, and evaluated the final results.

### Used Libraries

- **NumPy**: Provides a fast numerical array structure and helper functions.
- **pandas**: Provides a DataFrame structure to store data in memory and work with it easily and efficiently.
- **matplotlib**: The essential Machine Learning package in Python.
- **sklearn**: Basic plotting library in Python; most other Python plotting libraries are built on top of it.
- **seaborn**: Advanced statistical plotting library.
- **pycaret**: Offers streamlined workflows and a wide range of pre-built algorithms and techniques to experiment with different models and compare their performance using different evaluation metrics.


## Data analysis

We start by importing the required libraries and plotting some graphs for initial analysis of the dataset.

### Key Statistics

- Win-Loss Record: This is the most straightforward indicator. Teams with more wins are more likely to make the playoffs. Historically, teams with around a .500 or better win-loss record tend to have a good chance of making the playoffs.

- Winning Percentage: Similar to win-loss record, winning percentage (Wins / Total Games) is a fundamental metric used to assess a team's performance.

- Points Per Game (PPG): Teams that score more points on average are often more successful. This statistic reflects a team's offensive efficiency.

- Points Allowed Per Game (PAPG): Teams that allow fewer points per game have a stronger defense. Defensive efficiency is a critical factor in determining playoff success.

- Net Rating: Net rating is the difference between a team's offensive rating (points scored per 100 possessions) and their defensive rating (points allowed per 100 possessions). Teams with positive net ratings are often playoff-bound.

- Field Goal Percentage (FG%): Shooting efficiency is a key factor in a team's offensive performance. A high field goal percentage indicates effective shooting.

- Three-Point Percentage (3P%): The ability to make three-point shots is crucial in modern basketball. Teams with high three-point percentages often perform well.

- Free Throw Percentage (FT%): Teams with good free throw shooting can close out close games more effectively.

- Rebounds Per Game (RPG): Rebounding is a key component of both offense and defense. Teams that dominate the boards tend to have an advantage.

- Assists Per Game (APG): Ball movement and sharing are critical in the NBA. Teams with high assist numbers often have a strong offense.

- Steals and Blocks: Defensive statistics such as steals and blocks indicate a team's ability to disrupt the opponent's offense and protect the rim.

- Turnovers: Reducing turnovers is important for maintaining possession and minimizing scoring opportunities for the opposition.

In [ ]:
import data_manip
import os
import warnings
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from enum import Enum
import seaborn as sb
from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#from pycaret.classification import *
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%pip install imbalanced-learn


# Set the warning filter to "ignore"
warnings.filterwarnings("ignore", category=UserWarning)

awards = pd.read_csv('modified_data/awards_players.csv', na_values=['NA'], delimiter=",")
coaches = pd.read_csv('modified_data/coaches.csv', na_values=['NA'], delimiter=",")
players = pd.read_csv('modified_data/players.csv', na_values=['NA'], delimiter=",")
players_teams = pd.read_csv('modified_data/players_teams.csv', na_values=['NA'], delimiter=",")
series_post = pd.read_csv('modified_data/series_post.csv', na_values=['NA'], delimiter=",")
teams = pd.read_csv('modified_data/teams.csv', na_values=['NA'], delimiter=",")
teams_post = pd.read_csv('modified_data/teams_post.csv', na_values=['NA'], delimiter=",")


print(teams)

In [ ]:
# Custom formatting function
def custom_format(value):
    # Check if the value is a number (int or float)
    if isinstance(value, (int, float)):
        # If it's an integer, format as an integer
        if isinstance(value, int):
            return value
        # If it's a float, format with 2 decimal places
        elif isinstance(value, float):
            return round(value, 2)
    else:
        return value

formatted_df = teams.applymap(custom_format)
formatted_df.describe()

Playoff appearences by team

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

teams_df = pd.read_csv('./modified_data/teams.csv')

playoff_teams = teams_df[teams_df['playoff'] == 'Y']

playoff_count = playoff_teams['name'].value_counts().reset_index()
playoff_count.columns = ['Team', 'Playoff Appearances']

playoff_count = playoff_count.sort_values(by='Playoff Appearances', ascending=False)
print(playoff_count)

plt.figure(figsize=(10, 6))
plt.bar(playoff_count['Team'], playoff_count['Playoff Appearances'])
plt.xlabel('Team')
plt.ylabel('Playoff Appearances')
plt.title('Playoff Appearances per Team')
plt.xticks(rotation=90) 
plt.tight_layout()

plt.show()

Average players age per team

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

players_df = pd.read_csv('./modified_data/players.csv')
players_teams_df = pd.read_csv('./modified_data/players_teams.csv')
teams_df = pd.read_csv('./modified_data/teams.csv')

players_df = players_df[~players_df['birthDate'].str.startswith('0000')]

players_df['birthYear'] = players_df['birthDate'].str[:4].astype(int)
players_df['idade'] = 2023 - players_df['birthYear']

players_teams_df = players_teams_df[players_teams_df['year'] == 10]

merged_df = players_teams_df.merge(players_df, left_on='playerID', right_on='bioID')

team_avg_age = merged_df.groupby('tmID')['idade'].mean().reset_index()

team_avg_age = team_avg_age.merge(teams_df, left_on='tmID', right_on='tmID')

plt.figure(figsize=(10, 6))
plt.bar(team_avg_age['name'], team_avg_age['idade'])
plt.xlabel('Team')
plt.ylabel('Average age')
plt.title('Average players age per team (Year 10)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Win-Loss Record (Year 10)

In [ ]:
# Win-Loss Rate

import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("./modified_data/teams.csv")

data_year_10 = data[data['year'] == 10]

team_records = data_year_10.groupby('name')[['won', 'lost']].sum().reset_index()

team_records['win_loss_rate'] = team_records['won'] / (team_records['won'] + team_records['lost'])

team_records = team_records.sort_values(by='win_loss_rate', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(team_records['name'], team_records['win_loss_rate'])
plt.xlabel('Win-Loss Rate')
plt.ylabel('Team')
plt.title('Win-Loss Rate of All Teams in Year 10')
plt.grid(axis='x')

plt.show()

Winning Percentage last 10 years

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("./modified_data/teams.csv")

last_10_years = data[data['year'] >= data['year'].max() - 9]

team_records = last_10_years.groupby('name')[['won', 'lost']].sum().reset_index()

team_records['win_percentage'] = (team_records['won'] / (team_records['won'] + team_records['lost'])) * 100

team_records = team_records.sort_values(by='win_percentage', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(team_records['name'], team_records['win_percentage'])
plt.xlabel('Win Percentage (%)')
plt.ylabel('Team')
plt.title('Win Percentage of All Teams in the Last 10 Years')
plt.grid(axis='x')

plt.show()


Winning Percentage Year 10 only

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("./modified_data/teams.csv")

data_year_10 = data[data['year'] == 10]

team_records = data_year_10.groupby('name')[['won', 'lost']].sum().reset_index()

team_records['win_percentage'] = (team_records['won'] / (team_records['won'] + team_records['lost'])) * 100

team_records = team_records.sort_values(by='win_percentage', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(team_records['name'], team_records['win_percentage'], color='skyblue')
plt.xlabel('Win Percentage (%)')
plt.ylabel('Team')
plt.title('Win Percentage of All Teams in year 10')
plt.grid(axis='x')

plt.show()


Points per Game (PPG) Year 10

In [ ]:
import pandas as pd

df = pd.read_csv("./modified_data/teams.csv")

df= df[df['year'] == 10]

df['PPG'] = (df['o_pts'] / df['GP']).round()

result = df[['name', 'PPG']]
result.columns= ['Team', 'PointsPerGame']

result = result.sort_values(by='PointsPerGame', ascending=False)

print(result)


Net Rating (Year 10)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("./modified_data/teams.csv")

data_year_10 = data[data['year'] == 10]

team_records = data_year_10.groupby('name')[['o_fgm', 'o_fga']].sum().reset_index()

team_records['percentage'] = ((team_records['o_fgm']) / (team_records['o_fga']))* 100

team_records = team_records.sort_values(by='percentage', ascending=False)

plt.figure(figsize=(12, 8))
plt.barh(team_records['name'], team_records['percentage'], color='skyblue')
plt.xlabel('Percentage (%)')
plt.ylabel('Team')
plt.title('Net Rating (Year 10)')
plt.grid(axis='x')

plt.show()


Field Goal Percentage (FG%) Year 10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['FG%'] = (df['o_fgm'] / df['o_fga']) * 100

fg_percentages = df.groupby('name')['FG%'].mean().reset_index()
result = fg_percentages.sort_values(by='FG%', ascending=False)


print(result)



Three Points Percentage (FT%) Year 10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['3P%'] = (df['o_3pm'] / df['o_3pa']) * 100

fg_percentages = df.groupby('name')['3P%'].mean().reset_index()
result = fg_percentages.sort_values(by='3P%', ascending=False)

print(result)


Free Throw Percentage (FT%)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['FT%'] = (df['o_ftm'] / df['o_fta']) * 100

fg_percentages = df.groupby('name')['FT%'].mean().reset_index()
result = fg_percentages.sort_values(by='FT%', ascending=False)

print(result)


Rebounds per Game (RPG) Year 10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['Rebounds'] = (df['o_reb'] + df['d_reb'])/ df['GP']

fg_percentages = df.groupby('name')['Rebounds'].mean().reset_index()
result = fg_percentages.sort_values(by='Rebounds', ascending=False)

print(result)


Assists per Game (APG) Year 10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['Assists'] = (df['o_asts'])/ df['GP']

fg_percentages = df.groupby('name')['Assists'].mean().reset_index()
result = fg_percentages.sort_values(by='Assists', ascending=False)

print(result)


Steals and Blocks per Game (Year 10)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['Steals and Blocks'] = (df['d_stl'] + df['d_blk']) / df['GP']

fg_percentages = df.groupby('name')['Steals and Blocks'].mean().reset_index()
result = fg_percentages.sort_values(by='Steals and Blocks', ascending=False)

print(result)


Turnovers (Year 10)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("./modified_data/teams.csv")

df = df[df['year'] == 10]

df['Turnovers'] = (df['d_to']) / df['GP']

fg_percentages = df.groupby('name')['Turnovers'].mean().reset_index()
result = fg_percentages.sort_values(by='Turnovers', ascending=False)

print(result)


All tables together

In [12]:
import pandas as pd

file_paths = ["./modified_data/teams.csv"] 

columns_to_use = ["name", "d_to", "GP", "d_stl", "d_blk", "o_asts", "o_reb", "d_reb"]  

all_turnovers = []
all_steals_blocks = []
all_assists = []
all_rebounds = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    df = df[df['year'] == 10]
    
    if 'd_to' in columns_to_use:
        df['Turnovers'] = (df['d_to']) / df['GP']
        turnovers_percentages = df.groupby('name')['Turnovers'].mean().reset_index()
        all_turnovers.append(turnovers_percentages)
    
    if 'd_stl' in columns_to_use or 'd_blk' in columns_to_use:
        df['Steals and Blocks'] = (df['d_stl'] + df['d_blk']) / df['GP']
        steals_blocks_percentages = df.groupby('name')['Steals and Blocks'].mean().reset_index()
        all_steals_blocks.append(steals_blocks_percentages)
    
    if 'o_asts' in columns_to_use:
        df['Assists'] = (df['o_asts']) / df['GP']
        assists_percentages = df.groupby('name')['Assists'].mean().reset_index()
        all_assists.append(assists_percentages)

    if 'o_reb' in columns_to_use or 'd_reb' in columns_to_use:
        df['Rebounds'] = (df['o_reb'] + df['d_reb']) / df['GP']
        rebounds_percentages = df.groupby('name')['Rebounds'].mean().reset_index()
        all_rebounds.append(rebounds_percentages)

final_turnovers_df = pd.concat(all_turnovers, ignore_index=True)
final_steals_blocks_df = pd.concat(all_steals_blocks, ignore_index=True)
final_assists_df = pd.concat(all_assists, ignore_index=True)
final_rebounds_df = pd.concat(all_rebounds, ignore_index=True)

final_combined_df = final_turnovers_df.merge(final_steals_blocks_df, on='name', how='outer')
final_combined_df = final_combined_df.merge(final_assists_df, on='name', how='outer')
final_combined_df = final_combined_df.merge(final_rebounds_df, on='name', how='outer')

final_combined_df.to_csv("combined_data.csv", index=False)

print(final_combined_df)


                        name  Turnovers  Steals and Blocks    Assists   
0              Atlanta Dream  17.676471          14.117647  16.088235  \
1                Chicago Sky  14.382353          11.588235  15.647059   
2            Connecticut Sun  16.147059          10.970588  17.882353   
3              Detroit Shock  15.617647          11.441176  16.911765   
4              Indiana Fever  18.764706          11.382353  15.500000   
5         Los Angeles Sparks  13.617647          12.205882  17.264706   
6             Minnesota Lynx  15.970588          11.470588  14.970588   
7           New York Liberty  16.117647          11.558824  15.470588   
8            Phoenix Mercury  13.529412          11.852941  18.382353   
9        Sacramento Monarchs  16.941176          12.029412  15.882353   
10  San Antonio Silver Stars  14.764706          11.411765  17.588235   
11             Seattle Storm  16.323529          11.941176  15.764706   
12        Washington Mystics  16.941176          13

#### Checking for null values

In [ ]:
dataset.isna().sum()

### Data Encoding

In [ ]:
data_encoded = dataset.copy()

# Combine "AirportFrom" and "AirportTo" columns to create a unified set of airports
airports = pd.concat([data_encoded['AirportFrom'], data_encoded['AirportTo']])

# Perform label encoding on the combined set of airports
label_encoder_airports = LabelEncoder()
encoded_airports = label_encoder_airports.fit_transform(airports)

# Perform label encoding on the "Airline" column
label_encoder_airline = LabelEncoder()
data_encoded['Airline'] = label_encoder_airline.fit_transform(data_encoded['Airline'])

# Update "AirportFrom" and "AirportTo" columns with the encoded values
data_encoded['AirportFrom'] = encoded_airports[:len(data_encoded)]
data_encoded['AirportTo'] = encoded_airports[len(data_encoded):]

# Remove the "Flight" column
data_encoded = data_encoded.drop('Flight', axis=1)

#### Checking the correlation of values

In [ ]:
dataset_corr = data_encoded.corr()
plt.figure(figsize=(20,20))
mask = np.zeros_like(dataset_corr)
mask[np.triu_indices_from(mask, k=1)] = True
sb.heatmap(dataset_corr, cmap="YlGnBu", annot=True, square=True, mask=mask, fmt='.2f', annot_kws={"size": 10});
plt.show()

### All Data Histogram

In [ ]:
data_encoded.hist(bins=30, figsize=(30, 16), sharey=False)

- Relation between the day of the week and delay in flights

In [ ]:
sb.catplot(x="DayOfWeek", kind="count", data=data_encoded, hue="Class")

- Relation between the airline and delay in flights

In [ ]:
sb.catplot(x="Airline", kind="count", data=data_encoded, hue="Class")

### The dataset is slightly imbalanced

In [ ]:
fraud_df_pie_chart = data_encoded.loc[data_encoded['Class'] == 1.0]
not_fraud_df_pie_chart = data_encoded.loc[data_encoded['Class'] == 0.0]

array_pie_chart = np.array([len(fraud_df_pie_chart), len(not_fraud_df_pie_chart)])
pie_chart_labels = ["Delayed", "Not Delayed"]

plt.pie(array_pie_chart, labels=pie_chart_labels, autopct='%.2f')
plt.show()

## Data Preprocessing

After examining the dataset and assessing its characteristics, we conducted a comprehensive analysis. The results revealed a high level of data consistency, with no missing values or notable outliers observed. As a consequence, the dataset demonstrated a remarkable level of readiness for analysis, requiring minimal data preprocessing efforts.

In [ ]:
data_encoded.info()

### Train and Test split data

Dividimos os dados em conjunto de input e label para os classificadores do Scikit. Label é a coluna Class and input é as restantes colunas

In [ ]:
data_encoded['Class'] = data_encoded['Class'].astype('category')

col_names = list(data_encoded.columns)
col_names.remove('Class')

inputs = data_encoded[col_names].values
labels = data_encoded['Class'].values

Resumidamente dividi dados em dados de teste e treinamento, mantendo a mesma distribuição das classes inicias, usando 1/4 do dataset original

- stratify - para manter a distribuição de classes 
- train_in - variável que armazena as características de treinamento
- test_in - variável que armazena as características de teste
- train_classes - armazena as classes dos dados de treinamento
- test_classes - armazena as classes dos dados de testes
- random_state - para garantir randomness na divisão de dados


In [ ]:
from sklearn.model_selection import train_test_split

(train_in,
 test_in,
 train_classes,
 test_classes) = train_test_split(inputs, labels, test_size=0.25, random_state=1, stratify=labels)


The data analyses showed us that our working dataset is unbalanced. We implemented both undersampling and oversampling. Undersampling removes samples from majority categories, while oversampling duplicates samples from minority categories. Generally oversampling is preferred

Conta a ocorrência de cada classe nos conjuntos de treinamento e teste. No training set, tem 224338 de 0 e 180198 de 1. Da um overview da distribuição das classes em cada conunto

In [ ]:
from collections import Counter

print("---Train Set---")
print(Counter(train_classes))
print("\n---Test Set---")
print(Counter(test_classes))

É usado para balancear os dados. Remove aleatoriamente dados da classe com maior número de dados até que se encontro o equilíbrio que se quer. Undersampling.

In [ ]:

from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()

us_inputs, us_labels = rus.fit_resample(train_in, train_classes)

print(Counter(us_labels))

Faz oversampling dos dados e conseguimos ver que realmente os dados ficaram equilibradoos nos dados de treino. aumenta o número de linhas, tornando-a mais proporcionsl à classe maioritaria<>

In [ ]:
from imblearn.over_sampling import SMOTE

ros = SMOTE()

os_inputs, os_labels = ros.fit_resample(train_in, train_classes)

print(Counter(os_labels))

We used a StandardScaler from SciKit Learn's preprocessing library to standardize the data. Porque é necessário para o K nearest neighbrs e o SVM

Fas-se a padronização dos dados para garantir que as características contribuam igualmente para os modelos de machine learning, e evita que uma caraterística em particular dominee o processo de learning da outra

In [ ]:
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()

scaler.fit(train_in)
train_in = scaler.fit_transform(train_in)
test_in = scaler.fit_transform(test_in)

scaler.fit(os_inputs)
os_inputs = scaler.fit_transform(os_inputs)

scaler.fit(us_inputs)
us_inputs = scaler.fit_transform(us_inputs)

# Classification

## K-Nearest Neighbors

### Original Dataset

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

classifier = KNeighborsClassifier()
classifier.fit(train_in, train_classes)
y_pred = classifier.predict(test_in)

result = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print (result1) 

knn_og_report = classification_report(test_classes, y_pred,output_dict=True)

### Undersampled Dataset

In [ ]:
classifier = KNeighborsClassifier()
classifier.fit(us_inputs, us_labels)
y_pred = classifier.predict(test_in)

knn_confusion_matrix = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(knn_confusion_matrix)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print(result1)

knn_us_report = classification_report(test_classes, y_pred, output_dict=True)

### Oversampled Dataset

In [ ]:
classifier = KNeighborsClassifier()
classifier.fit(os_inputs, os_labels)
y_pred = classifier.predict(test_in)

result = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print (result1)

knn_os_report = classification_report(test_classes, y_pred,output_dict=True)

## Decision Tree Classifier

### Original Dataset

Confusion matrix:
TP FP
TN FN
FP = False Positive - deu que iam atrasar (1) mas na realidade é 0

Precision - mede a proporção de dados corretamente calculado TP/(TP+FP)

Accuracy - mede a correção no geral (TP + TN) / (TP + TN + FP + FN).
    - em dados não balenciados pode ser misleading

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dtc.fit(train_in, train_classes)
dtc_prediction = dtc.predict(test_in)

dtc_classification_report = classification_report(test_classes, dtc_prediction, output_dict=True)

print("--- Original dataset ---\n")
print("Confusion matrix:")
print(f"{confusion_matrix(test_classes, dtc_prediction)}\n")
print(f"Classification report:")
print(f"{classification_report(test_classes, dtc_prediction)}\n")


### Undersampled Dataset

In [ ]:
dtc.fit(us_inputs, us_labels)
dtc_prediction = dtc.predict(test_in)

dtc_us_classification_report = classification_report(test_classes, dtc_prediction, output_dict=True)

print("--- Undersampled dataset ---\n")
print(f"Confusion matrix:\n{confusion_matrix(test_classes, dtc_prediction)}\n")
print(f"Classification report:\n{classification_report(test_classes, dtc_prediction)}\n")

### Oversampled Data

In [ ]:
dtc.fit(os_inputs, os_labels)
dtc_prediction = dtc.predict(test_in)

dtc_os_classification_report = classification_report(test_classes, dtc_prediction, output_dict=True)

print("--- Oversampled dataset ---\n")
print(f"Confusion matrix:\n{confusion_matrix(test_classes, dtc_prediction)}\n")
print(f"Classification report:\n{classification_report(test_classes, dtc_prediction)}\n")

## SVM

### Original Dataset

In [ ]:
svc = SVC()

svc.fit(train_in, train_classes)
y_pred = svc.predict(test_in)

result = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print (result1) 

svc_og_report = classification_report(test_classes, y_pred,output_dict=True)

### Undersampled Dataset

In [ ]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(us_inputs, us_labels)
y_pred = svc.predict(test_in)

result = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print (result1) 

svc_us_report = classification_report(test_classes, y_pred,output_dict=True)

### Oversampled Dataset

In [ ]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(os_inputs, os_labels)
y_pred = svc.predict(test_in)

result = confusion_matrix(test_classes, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(test_classes, y_pred)
print("Classification Report:",)
print (result1) 

svc_os_report = classification_report(test_classes, y_pred,output_dict=True)

## Result Analyses

### All algorithms

In [ ]:

all_algorithms_data = pd.read_csv('models_comparison.csv', na_values=['NA'], delimiter=",")
all_algorithms_data.set_index("Model", inplace=True)

sb.heatmap(all_algorithms_data, cmap="YlGnBu", annot=True)
plt.xlabel('Results')
plt.ylabel('ML Models')
plt.show()


### Selected algorithms

In [ ]:

selected_algorithms_data = pd.read_csv('models_comparison_selected.csv', na_values=['NA'], delimiter=",")
selected_algorithms_data.set_index("Model", inplace=True)

sb.heatmap(selected_algorithms_data, cmap="YlGnBu", annot=True)
plt.xlabel('Results')
plt.ylabel('ML Models')
plt.show()


### Comparing Accuracys of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['Accuracy'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='Accuracy', data=selected_algorithms_data, color='#A7226E')

# Set the plot title and axis labels
plt.title('Accuracy Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Accuracy')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing AUC (Area Under Curve) of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['AUC'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='AUC', data=selected_algorithms_data, color='#FE4365')

# Set the plot title and axis labels
plt.title('AUC Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('AUC')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing Recall of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['Recall'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='Recall', data=selected_algorithms_data, color='#9DE0AD')

# Set the plot title and axis labels
plt.title('Recall Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Recall')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing Precision of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['Precision'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='Precision', data=selected_algorithms_data, color='#F7DB4F')

# Set the plot title and axis labels
plt.title('Precision Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Precision')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing F1-score of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['F1'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='F1', data=selected_algorithms_data, color='#F26B38')

# Set the plot title and axis labels
plt.title('F1-Score Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('F1-Score')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing Kappa of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['Kappa'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='Kappa', data=selected_algorithms_data, color='#2F9599')

# Set the plot title and axis labels
plt.title('Kappa Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Kappa')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing Matthews Correlation Coefficient of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['MCC'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='MCC', data=selected_algorithms_data, color='#FF4E50')

# Set the plot title and axis labels
plt.title('Matthews Correlation Coefficient Comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Matthews Correlation Coefficient')

plt.xticks(rotation=90)

# Display the plot
plt.show()

### Comparing Training Time (sec) of each algorythm

In [ ]:
selected_algorithms_data = selected_algorithms_data.sort_values(by=['TT (Sec)'], ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sb.barplot(x=selected_algorithms_data.index, y='TT (Sec)', data=selected_algorithms_data, color='#9DE0AD')

# Set the plot title and axis labels
plt.title('Training Time comparison by Algorithm')
plt.xlabel('Model')
plt.ylabel('Training Time (Sec)')

plt.xticks(rotation=90)

# Display the plot
plt.show()